# CS211: Data Privacy
## Final Project
### Nikhil and James

In [93]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import random
plt.style.use('seaborn-whitegrid')

# Preliminary utility functions
def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(qs, sensitivity, epsilon):
    return [laplace_mech(q, sensitivity, epsilon) for q in qs]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

# Database containing the default data of credit card clients Taiwan during 2005 
cc_default_db = pd.read_csv('https://raw.githubusercontent.com/nikhilchoppa/cs211-final-project/main/UCI_Credit_Card.csv')

/tmp/ipykernel_52444/1109455332.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [94]:
cc_default_db.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


### There are 25 variables in the `cc_default_db` Dataset:

- ID: ID of each client
- LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit
- SEX: Gender (1=male, 2=female)
- EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
- MARRIAGE: Marital status (1=married, 2=single, 3=others)
- AGE: Age in years
- PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, … 8=payment delay for eight months, 9=payment delay for nine months and above)
- PAY_2: Repayment status in August, 2005 (scale same as above)
- PAY_3: Repayment status in July, 2005 (scale same as above)
- PAY_4: Repayment status in June, 2005 (scale same as above)
- PAY_5: Repayment status in May, 2005 (scale same as above)
- PAY_6: Repayment status in April, 2005 (scale same as above)
- BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar)
- BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar)
- BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar)
- BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar)
- BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar)
- BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar)
- PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar)
- PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar)
- PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar)
- PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar)
- PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar)
- PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar)
- default.payment.next.month: Default payment (1=yes, 0=no)

## Generalize ages
A good first step to ensure privacy but only will satisfy k-anonimity, differential privacy will require additonal work 

In [95]:
# Generalizing the given column to essentially round to the nth decimal place
depths = {
    'AGE': 1
}

# Grabbed this part from HW2 / the textbook, applies the rounding described above to the given column(s) in the dataset
cc_default_db[['AGE']].apply(lambda x: x.apply(lambda y: int(int(y/(10**depths[x.name]))*(10**depths[x.name]))))

,AGE
0,20
1,20
2,30
3,30
4,50
...,...
29995,30
29996,40
29997,30
29998,40


## Privacy Budget
Define the privacy budget in terms of epsilon $\epsilon$, keeping in mind that each row is an individual and allocating the budget in terms of the individual

In [96]:
epsilon = 1

Adding Differential Privacy to `AGE, LIMIT_BAL, BILL_AMT, PAY_AMT`

In [97]:
# preserves epsilon-differential privacy
def above_threshold(queries, df, T, epsilon):
    T_hat = T + np.random.laplace(loc=0, scale = 2/epsilon)   
    for idx, q in enumerate(queries):
        nu_i = np.random.laplace(loc=0, scale = 4/epsilon)
        if q(df) + nu_i >= T_hat:
            return idx
    # if the algorithm "fails", return a random index 
    # more convenient in certain use cases    
    return random.randint(0,len(queries)-1)

In [100]:
# Applying the sparse vector technique to our differentially private statistics
# This will have to be done for each column in BILL_AMT1-6 and PAY_AMT1-6
dp_col_stats = {}
for col in cc_default_db.columns:
    if 'BILL_AMT' in col or 'PAY_AMT' in col:
        df = cc_default_db[col]
        # Construct the stream of queries
        bs = range(1,150,5)
        queries = [lambda df: df.clip(lower=0, upper=b).sum() - df.clip(lower=0, upper=b+1).sum() for b in bs]

        # Run AboveThreshold, using 1/3 of the privacy budget, to find a good clipping parameter
        epsilon_svt = epsilon / 3
        final_b = bs[above_threshold(queries, df, 0, epsilon_svt)]

        # Compute the noisy sum and noisy count, using 1/3 of the privacy budget for each
        epsilon_sum = epsilon / 3
        epsilon_count = epsilon / 3

        noisy_sum = laplace_mech(df.clip(lower=0, upper=final_b).sum(), final_b, epsilon_sum)
        noisy_count = laplace_mech(len(df), 1, epsilon_count)
        noisy_average = noisy_sum/noisy_count
        
        # Add those statistics to the respective column
        dp_col_stats[col] = [noisy_sum, noisy_count, noisy_average]
dp_col_stats

{'BILL_AMT1': [2354982.330819592, 29999.03882958853, 78.5019261516084],
 'BILL_AMT2': [563227.3991367755, 30000.708905770563, 18.773803009316225],
 'BILL_AMT3': [1745683.1582616773, 29993.18815139449, 58.20265419768369],
 'BILL_AMT4': [3548061.4955348126, 30006.554416023264, 118.24288274964938],
 'BILL_AMT5': [3507393.2249496467, 30001.170929341883, 116.90854444348804],
 'BILL_AMT6': [910005.0497935981, 29999.022585245963, 30.334489972388443],
 'PAY_AMT1': [2969395.104007432, 30000.588073789066, 98.97789659002501],
 'PAY_AMT2': [1489196.6431823347, 29998.53981320909, 49.64230434064678],
 'PAY_AMT3': [1809243.7040941552, 30002.340474137192, 60.30341885006508],
 'PAY_AMT4': [2006911.1253805605, 29998.90623409327, 66.89947659157448],
 'PAY_AMT5': [1295630.2634974609, 29996.93796758164, 43.19208396862631],
 'PAY_AMT6': [2175694.2939372, 30002.200367348305, 72.51782426948358]}

## Calculating non DP statistics
In order to compare and show how adding noise affects the queries

In [103]:
# Counting queries for every row
col_counts = {}
for col in cc_default_db.columns:
    if col != 'ID': # Skip the ID column for obvious reasons
        col_counts[col] = cc_default_db[col].value_counts()

col_sums = {}
for col in cc_default_db.columns:
    if col != 'ID':
        col_sums[col] = cc_default_db[col].sum()

col_averages = {}
for col in cc_default_db.columns:
    if col != 'ID':
        col_averages[col] = cc_default_db[col].mean()
col_sums['BILL_AMT2']

1475372255.0

## Format database in order to calculate some statistics
### ...Merge all the columns that have `"_#"`
This is just signifying month to month statistics and it is going to be easier to work with if they are centralized to one column for PAY, BILL_AMT,and PAY_AMT
The time complexity here is a little unreasonable 

In [7]:
# Generate column names
merge_cols = []
for comb_col in ['PAY', 'BILL_AMT', 'PAY_AMT']:
    comb_row = []
    for i in range(1,7):
        if comb_col == 'PAY':
            comb_row.append(comb_col + '_' + str(i))
        else:
            comb_row.append(comb_col + str(i))
    merge_cols.append(comb_row)

In [8]:
fresh_cols = []
for col in merge_cols:
    new_row = []
    for row in range(len(cc_default_db)):
        new_col = []
        for sub_col in col:
            new_col.append(cc_default_db[sub_col][row])
        new_row.append(new_col)
    fresh_cols.append(new_row)

In [9]:
# Apply what we did above
for subcol in merge_cols:
    for c in subcol:
        cc_default_db.drop(c, axis=1, inplace=True)
cc_default_db['PAY'], cc_default_db['BILL_AMT'], cc_default_db['PAY_AMT'] = [fresh_cols[0], fresh_cols[1], fresh_cols[2]]

In [10]:
cc_default_db.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,default.payment.next.month,PAY,BILL_AMT,PAY_AMT
0,1,20000.0,2,2,1,24,1,"[2, 2, -1, -1, -2, -2]","[3913.0, 3102.0, 689.0, 0.0, 0.0, 0.0]","[0.0, 689.0, 0.0, 0.0, 0.0, 0.0]"
1,2,120000.0,2,2,2,26,1,"[-1, 2, 0, 0, 0, 2]","[2682.0, 1725.0, 2682.0, 3272.0, 3455.0, 3261.0]","[0.0, 1000.0, 1000.0, 1000.0, 0.0, 2000.0]"
2,3,90000.0,2,2,2,34,0,"[0, 0, 0, 0, 0, 0]","[29239.0, 14027.0, 13559.0, 14331.0, 14948.0, ...","[1518.0, 1500.0, 1000.0, 1000.0, 1000.0, 5000.0]"
3,4,50000.0,2,2,1,37,0,"[0, 0, 0, 0, 0, 0]","[46990.0, 48233.0, 49291.0, 28314.0, 28959.0, ...","[2000.0, 2019.0, 1200.0, 1100.0, 1069.0, 1000.0]"
4,5,50000.0,1,2,1,57,0,"[-1, 0, -1, 0, 0, 0]","[8617.0, 5670.0, 35835.0, 20940.0, 19146.0, 19...","[2000.0, 36681.0, 10000.0, 9000.0, 689.0, 679.0]"
